## Notebook 02: Fine-Tuning con Unsloth

Este notebook implementa el fine-tuning de un modelo Llama-3 usando LoRA (Low-Rank Adaptation) para crear el estilo de Alicia en el País de las Maravillas.

### 1. Instalación de herramientas 

**Nota:** Esta celda está diseñada para ejecutarse en **Google Colab**. Si trabajas en **local**, asegúrate de tener estas librerías ya instaladas en tu entorno Python.

⚠️ **Requisito:** Se necesita una **GPU con CUDA** para ejecutar este notebook. En Colab, usa un runtime con GPU (Runtime > Change runtime type > T4 GPU).

In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes

### 2. Importar librerías

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

### 3. Cargar modelo Llama-3 (optimizado para 4-bit)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

### 4. Configurar LoRA

Solo entrenamos el 1-2% de los parámetros usando LoRA (Low-Rank Adaptation).

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)

### 5. Formatear el Dataset

Formato Instrucción-Input-Respuesta (formato Alpaca) para el prompt engineering.

In [ ]:
prompt_format = """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
{}

### Entrada:
{}

### Respuesta:
{}"""

def formatting_prompts_func(examples):
    # The instruction is constant for all examples in the batch.
    constant_instruction = "Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas."
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    # Iterate over inputs and outputs, applying the constant instruction to each pair.
    for input_text, output_text in zip(inputs, outputs):
        text = prompt_format.format(constant_instruction, input_text, output_text)
        texts.append(text)
    return { "text" : texts, }

dataset = load_dataset("json", data_files="/content/synthetic_alicia.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

### 6. Entrenamiento

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer = tokenizer, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
    ),
)
trainer.train()

### 7. Exportar a GGUF para Ollama

In [ ]:
model.save_pretrained_gguf("model_alicia", tokenizer, quantization_method = "q4_k_m")

### 8. Cómo usar el modelo con Ollama

Una vez exportado el modelo, necesitas cargarlo en Ollama para usarlo.

#### Paso 1: Crear un Modelfile

Crea un archivo llamado `Modelfile` en tu directorio local con el siguiente contenido:

In [ ]:
%%writefile Modelfile
FROM ./model_alicia-Q4_K_M.gguf

TEMPLATE """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas.

### Entrada:
{{ .Prompt }}

### Respuesta:
"""

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER stop "###"

#### Paso 2: Importar el modelo a Ollama

Ejecuta este comando en tu terminal local (no en Colab):

In [ ]:
# En terminal local:
# ollama create alicia -f Modelfile

#### Paso 3: Usar el modelo

Desde terminal:

In [ ]:
# En terminal:
# ollama run alicia "Era una tarde soleada y los niños jugaban en el parque"

O desde Python con la API de Ollama:

In [ ]:
import requests

response = requests.post('http://localhost:11434/api/generate', 
    json={
        'model': 'alicia',
        'prompt': 'Era una tarde soleada y los niños jugaban en el parque',
        'stream': False
    }
)

print(response.json()['response'])